In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import numpy as np
import os
from dotenv import load_dotenv

In [26]:
base_csv_path = r".\TGE data\TGE_RTT_BASE.csv"
peak5_csv_path = r".\TGE data\TGE_RTT_PEAK.csv"

In [27]:
df_base_all = pd.read_csv(base_csv_path, sep=",", decimal=".", parse_dates=["Data"])
df_peak_all = pd.read_csv(peak5_csv_path, sep=",", decimal=".", parse_dates=["Data"])

In [28]:
base_last_date = df_base_all["Data"].tail(n=1).values[0]
peak_last_date = df_peak_all["Data"].tail(n=1).values[0]

if base_last_date != peak_last_date:
    raise Exception("Daty w plikach nie są sobie równe")
else:
    last_csv_date = pd.to_datetime(base_last_date, format="%Y-%m-%d")

In [29]:
yesterday = pd.to_datetime(dt.now().replace(hour=0, minute=0, second=0, microsecond=0))

In [30]:
diff = last_csv_date - yesterday

In [31]:
days = int((diff / np.timedelta64(1,"D")) * -1)

In [32]:
cur_day = last_csv_date + timedelta(days=1)
dates_to_get = []

for d in range(1, days):
    dates_to_get.append(cur_day.strftime("%d-%m-%Y"))
    cur_day = cur_day + timedelta(days=1)

In [33]:
df_contracts = pd.DataFrame({
    "Data" : [],
    "Kontrakt" : [],
    "tab_1" : [],
    "Kurs pierwszej transakcji (PLN/MWh)" : [],
    "DKR (PLN/MWh)" : [],
    "Kurs min. na sesji (PLN/MWh)" : [],
    "Kurs maks. na sesji (PLN/MWh)" : [],
    "Łączny wolumen obrotu (MWh)" : [],
    "Liczba kontraktów" : [],
    "Łączna wartość obrotu (PLN)": [],
    "Liczba transakcji" : [],
    "Łączna liczba otwartych pozycji LOP (MWh)" : []
})

df_base = df_contracts.copy()
df_peak = df_contracts.copy()

In [ ]:
load_dotenv()
DRIVER_PATH = os.getenv("WEBDRIVER_PATH")
driver_path = DRIVER_PATH # edgedriver could be downloaded here --> https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver
service = webdriver.EdgeService(executable_path=driver_path) # another example webdriver.ChromeService()

In [ ]:
driver = webdriver.Edge(service=service)
driver.get(rf"https://www.tge.pl/energia-elektryczna-otf?dateShow={dates_to_get[0]}&dateAction=prev")
driver.maximize_window()

Scraping function

In [ ]:
def get_contracts(contract_type: str, contract_df: pd.DataFrame, date) -> pd.DataFrame:

    date_container = driver.find_elements(By.TAG_NAME, "section")[-2]
    datepicker = date_container.find_element(By.ID, "datepicker")
    calendar_date = datepicker.get_attribute("value")
    datetime_calendar_date = dt.strptime(calendar_date, "%d-%m-%Y")
    excel_calendar_date = dt.strftime(datetime_calendar_date, "%Y-%m-%d")

    prices_container = driver.find_elements(By.TAG_NAME, "section")[-1]

    try:
        prices_container.find_element(By.CLASS_NAME, contract_type)
    except:
        print(f"contract not found for date {date}")
        return contract_df 
    
    contract_prices_container = prices_container.find_element(By.CLASS_NAME, contract_type)
    contracts_container = contract_prices_container.find_element(By.TAG_NAME, "tbody")
    contracts_info = contracts_container.find_elements(By.TAG_NAME, "tr")

    for contract in contracts_info:
        c_values = contract.find_elements(By.TAG_NAME, "td")

        day_contract = []

        for c_value in c_values:
            val = c_value.text
            if val == "-" or val == "":
                val = 0
                day_contract.append(val)
            else: day_contract.append(val)
        day_contract.insert(0, excel_calendar_date)
        contract_df.loc[len(contract_df)] = day_contract

    return contract_df


Scraping function invoke in a loop

In [ ]:
df_base = get_contracts("type-base", df_base, dates_to_get[0])
df_peak = get_contracts("type-peak5", df_peak, dates_to_get[0])

for (idx, date) in enumerate(dates_to_get[1:]):
    driver.get(rf"https://www.tge.pl/energia-elektryczna-otf?dateShow={date}&dateAction=prev")
    driver.implicitly_wait(7)
    df_base = get_contracts("type-base", df_base, date)
    df_peak = get_contracts("type-peak5", df_peak, date)

df_base.drop(columns=["tab_1"], axis=1, inplace=True)
df_peak.drop(columns=["tab_1"], axis=1, inplace=True)
driver.quit()

Data deduplication

In [38]:
df_base_all["key"] = df_base_all[[col for col in df_base_all.columns]].astype(str).agg("_".join, axis=1)
df_peak_all["key"] = df_peak_all[[col for col in df_peak_all.columns]].astype(str).agg("_".join, axis=1)
df_base["key"] = df_base[[col for col in df_base.columns]].astype(str).agg("_".join, axis=1)
df_peak["key"] = df_peak[[col for col in df_peak.columns]].astype(str).agg("_".join, axis=1)

df_base_b_dup = df_base.copy()
df_peak_b_dup = df_peak.copy()

df_base.drop_duplicates(keep="first", inplace=True)
df_peak.drop_duplicates(keep="first", inplace=True)

In [39]:
df_base.drop(columns=["key"], inplace=True)
df_peak.drop(columns=["key"], inplace=True)
df_base_all.drop(columns=["key"], inplace=True)
df_peak_all.drop(columns=["key"], inplace=True)

In [40]:
def save_in_csv(df: pd.DataFrame, path: str):
    df.to_csv(
        path_or_buf=path,
        header=False,
        index=False,
        sep=",",
        mode="a",
        encoding="utf-8",
        float_format="."
    )

In [41]:
save_in_csv(df_base, base_csv_path)
save_in_csv(df_peak, peak5_csv_path)